#### **하이브리드 검색 시스템**
이 코드는 AI 관련 뉴스 기사를 효과적으로 검색하기 위한 하이브리드 검색 시스템을 구현한 것입니다. 벡터 기반 의미론적 검색과 키워드 기반 검색을 결합하여 더 정확한 검색 결과를 제공합니다.

##### **주요 기능**
1. 문서 처리
    - JSON 형식의 뉴스 데이터 로드
    - 문서를 청크 단위로 분할
    - 벡터 DB 및 키워드 검색용 인덱스 생성

2. 하이브리드 검색
    - 벡터 기반 의미론적 검색 (FAISS)
    - 키워드 기반 검색 (BM25)
    - 두 검색 방식의 결과를 가중치를 적용하여 통합

3. 데이터 관리
    - 벡터 스토어 저장/로드
    - 처리된 문서 데이터 저장/로드
    - 진행 상황 로깅

##### **검색 가중치 설정 가이드**
- 의미론적 검색 중심 (semantic_weight=0.7)
    - 문맥과 의미를 더 중요하게 고려
    - 유사한 주제의 문서도 검색 가능
    - 예: "AI 기술의 미래 전망" → AI 발전 방향, 기술 트렌드 등 관련 문서 포함

- 키워드 검색 중심 (semantic_weight=0.3)
    - 정확한 키워드 매칭을 중시
    - 특정 용어나 개념이 포함된 문서 우선
    - 예: "삼성전자 AI 칩" → 정확히 해당 키워드가 포함된 문서 우선

- 균형잡힌 검색 (semantic_weight=0.5)
    - 두 방식의 장점을 균형있게 활용
    - 일반적인 검색에 적합
    - 예: "자율주행 안전" → 키워드 매칭과 의미적 연관성 모두 고려

In [34]:
import os
import json
import pickle
from typing import List, Dict, Tuple
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
from rank_bm25 import BM25Okapi
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [35]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")

In [52]:
class AIBooksRAG:
    """
    JSON 데이터를 활용한 Hybrid RAG 시스템
    - LangChain의 FAISS 벡터스토어를 이용한 의미론적 검색
    - BM25를 이용한 키워드 기반 검색
    - 하이브리드 검색 기능 제공
    """

    def __init__(self):
        self.vector_store = None
        self.metadata = []
        self.bm25 = None
        self.embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)

    # 1. 임베딩 생성 함수
    def get_embedding(self, text: str) -> List[float]:
        """텍스트를 OpenAI 임베딩 모델로 임베딩"""
        return self.embeddings.embed_query(text)

    # 2. 목차 파싱 함수
    def parse_toc(self, toc_html: str) -> List[Dict[str, List[str]]]:
        """HTML 형태의 목차 데이터를 파싱하여 계층적 구조로 반환"""
        soup = BeautifulSoup(toc_html, "html.parser")
        chapters = [b.get_text() for b in soup.find_all("b")]
        items = [br.next_sibling.strip() for br in soup.find_all("br") if br.next_sibling]

        structured_toc = []
        current_chapter = None

        for item in items:
            if item in chapters:
                current_chapter = item
                structured_toc.append({"chapter": current_chapter, "items": []})
            elif current_chapter:
                structured_toc[-1]["items"].append(item)

        return structured_toc

    # 3. 데이터 임베딩 및 문서 생성 함수
    def create_documents(self, book_data: List) -> List[Document]:
        """책 데이터를 LangChain Document로 변환"""
        documents = []
        
        for book in book_data:
            # 책 기본 정보
            documents.append(Document(
                page_content=book["title"],
                metadata={
                    "type": "title",
                    "author": book["author"],
                    "pubDate": book["pubDate"],
                    "categoryName": book["categoryName"]
                }
            ))
            documents.append(Document(
                page_content=book["description"],
                metadata={
                    "type": "description",
                    "author": book["author"],
                    "pubDate": book["pubDate"],
                    "categoryName": book["categoryName"]
                }
            ))

            # 목차 정보
            toc_html = book["toc"]
            structured_toc = self.parse_toc(toc_html)
            for chapter in structured_toc:
                for item in chapter["items"]:
                    item_with_context = f"{chapter['chapter']} - {item}"
                    documents.append(Document(
                        page_content=item_with_context,
                        metadata={
                            "type": "toc",
                            "chapter": chapter["chapter"],
                            "item": item,
                            "author": book["author"],
                            "pubDate": book["pubDate"],
                            "categoryName": book["categoryName"]
                        }
                    ))
        return documents

    # 4. JSON 데이터 로드 함수
    def load_json_files(self, directory: str) -> List[Dict]:
        """지정된 디렉토리에서 모든 JSON 파일을 읽어 책 정보 리스트 반환"""
        data = []
        for filename in os.listdir(directory):
            if filename.endswith(".json"):
                filepath = os.path.join(directory, filename)
                with open(filepath, "r", encoding="utf-8") as f:
                    data.append(json.load(f))
        return data

    # 5. FAISS 벡터스토어 생성
    def create_vector_store(self, json_dir: str, vector_store_path: str):
        """JSON 데이터를 읽고 FAISS 벡터스토어 생성 및 저장"""
        book_data_list = self.load_json_files(json_dir)
        documents = []
        for book_data in book_data_list:
            documents.extend(self.create_documents(book_data))

        # LangChain FAISS 벡터스토어 생성
        vector_store = FAISS.from_documents(documents, self.embeddings)
        vector_store.save_local(vector_store_path)
        self.vector_store = vector_store
        self.metadata = documents

    # 6. FAISS 벡터스토어 로드
    def load_vector_store(self, vector_store_path: str):
        """FAISS 벡터스토어 및 메타데이터 로드"""
        self.vector_store = FAISS.load_local(
            vector_store_path,
            embeddings=self.embeddings,
            allow_dangerous_deserialization=True
        )
        self.metadata = [doc.metadata for doc in self.vector_store.similarity_search("", k=5)]

    # 7. BM25 초기화
    def initialize_bm25(self):
        """BM25 검색 엔진 초기화"""
        tokenized_corpus = [doc.page_content.lower().split() for doc in self.metadata]
        self.bm25 = BM25Okapi(tokenized_corpus)

    # 8. Hybrid 검색
    def hybrid_search(self, query: str, k: int = 5, semantic_weight: float = 0.5) -> List[Tuple[str, float]]:
        """FAISS 및 BM25를 결합한 하이브리드 검색"""
        # FAISS 검색
        faiss_results = self.vector_store.similarity_search_with_score(query, k=k)

        # BM25 검색
        tokenized_query = query.lower().split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        bm25_results = sorted(
            [(self.metadata[i], bm25_scores[i]) for i in range(len(self.metadata))],
            key=lambda x: x[1],
            reverse=True
        )[:k]

        # 하이브리드 결합
        combined_scores = {}
        for doc, score in faiss_results:
            combined_scores[doc.metadata["type"]] = semantic_weight * (1 - score)
        for doc, score in bm25_results:
            if doc["type"] in combined_scores:
                combined_scores[doc["type"]] += (1 - semantic_weight) * score
            else:
                combined_scores[doc["type"]] = (1 - semantic_weight) * score

        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
        return sorted_results[:k]


In [53]:
# AIBooksRAG 클래스 초기화
rag = AIBooksRAG()

# JSON 데이터 디렉토리 및 저장 경로
json_dir = "./books"
vector_store_path = "./books_vectorstore"

# 벡터스토어 생성
rag.create_vector_store(json_dir, vector_store_path)

# 벡터스토어 로드
rag.load_vector_store(vector_store_path)

# BM25 초기화
rag.initialize_bm25()

# 하이브리드 검색
query = "고등학교참고서"
results = rag.hybrid_search(query, k=5)
print("\n=== Hybrid Search Results ===")
for doc_type, score in results:
    print(f"Type: {doc_type}, Score: {score:.4f}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embedding

AttributeError: 'dict' object has no attribute 'page_content'

In [ ]:
# 임베딩 API 호출 함수
def get_embedding(text: str, model: str = OPENAI_EMBEDDING_MODEL) -> List[float]:
    """텍스트를 주어진 임베딩 모델로 임베딩"""
    response = requests.post(
        "https://api.openai.com/v1/embeddings",
        headers={"Authorization": f"Bearer {OPENAI_API_KEY}"},
        json={"model": model, "input": text},
    )
    response.raise_for_status()
    return response.json()["data"][0]["embedding"]

# 목차 데이터 파싱 함수
def parse_toc(toc_html: str) -> List[Dict[str, List[str]]]:
    """HTML 형태의 목차 데이터를 파싱하여 계층적 구조로 반환"""
    soup = BeautifulSoup(toc_html, "html.parser")
    chapters = [b.get_text() for b in soup.find_all("b")]
    items = [br.next_sibling.strip() for br in soup.find_all("br") if br.next_sibling]

    structured_toc = []
    current_chapter = None

    for item in items:
        # 큰 단원이 포함된 경우
        if item in chapters:
            current_chapter = item
            structured_toc.append({"chapter": current_chapter, "items": []})
        elif current_chapter:
            structured_toc[-1]["items"].append(item)

    return structured_toc

# 데이터 임베딩 함수
def embed_book_data(book_data: Dict, model: str = OPENAI_EMBEDDING_MODEL):
    """책 데이터를 계층적으로 임베딩"""
    # 책 기본 정보 임베딩
    title_embedding = get_embedding(book_data["title"], model=model)
    description_embedding = get_embedding(book_data["description"], model=model)
    category_embedding = get_embedding(book_data["categoryName"], model=model)

    # 목차 임베딩
    toc_html = book_data["toc"]
    structured_toc = parse_toc(toc_html)

    toc_embeddings = []
    for chapter in structured_toc:
        chapter_title = chapter["chapter"]
        for item in chapter["items"]:
            # 문맥 포함하여 임베딩 생성
            item_with_context = f"{chapter_title} - {item}"
            item_embedding = get_embedding(item_with_context, model=model)
            toc_embeddings.append({
                "chapter": chapter_title,
                "item": item,
                "embedding": item_embedding
            })

    return {
        "title_embedding": title_embedding,
        "description_embedding": description_embedding,
        "category_embedding": category_embedding,
        "toc_embeddings": toc_embeddings,
    }

# JSON 파일 읽기 함수
def load_json_files(directory: str) -> List[Dict]:
    """지정된 디렉토리에서 모든 JSON 파일을 읽어 책 정보 리스트 반환"""
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                data.append(json.load(f))
    return data

# FAISS 인덱스 생성 및 저장
def create_faiss_vectorstore(json_dir: str, faiss_index_path: str, metadata_path: str):
    """
    JSON 데이터를 읽고 FAISS 인덱스와 메타데이터 저장
    """
    # JSON 파일 로드
    book_data_list = load_json_files(json_dir)[0]

    # 임베딩과 메타데이터 생성
    all_vectors = []
    metadata = []

    for book_data in book_data_list:
        embeddings = embed_book_data(book_data)
        # 벡터 추가
        all_vectors.append(embeddings["title_embedding"])
        all_vectors.append(embeddings["description_embedding"])
        all_vectors.append(embeddings["category_embedding"])

        # 메타데이터 추가
        metadata.append({
            "title": book_data["title"],
            "author": book_data["author"],
            "pubDate": book_data["pubDate"],
            "categoryName": book_data["categoryName"]
        })

        # 목차 벡터 추가
        for toc in embeddings["toc_embeddings"]:
            all_vectors.append(toc["embedding"])
            metadata.append({
                "title": book_data["title"],
                "chapter": toc["chapter"],
                "item": toc["item"]
            })

    # NumPy 배열로 변환
    all_vectors = np.array(all_vectors, dtype=np.float32)

    # FAISS 인덱스 생성
    dimension = all_vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(all_vectors)

    # 저장
    faiss.write_index(index, faiss_index_path)
    print(f"FAISS 인덱스 저장 완료: {faiss_index_path}")

    with open(metadata_path, "wb") as f:
        pickle.dump(metadata, f)
    print(f"메타데이터 저장 완료: {metadata_path}")

In [32]:
class AIBooksRAG:
    """
    AI 책 검색을 위한 RAG(Retrieval-Augmented Generation) 시스템
    
    이 클래스는 의미론적 검색과 키워드 기반 검색을 결합한 하이브리드 검색 기능을 제공합니다.

    Attributes:
        embeddings: OpenAI 임베딩 모델
        vector_store: FAISS 벡터 저장소
        bm25: 키워드 기반 검색을 위한 BM25 모델
        doc_mapping: 문서 ID와 문서 객체 간의 매핑
        logger: 로깅을 위한 로거 객체
    """
    def __init__(self, embedding_model):
        self.embeddings = embedding_model
        self.vector_store = None
        self.bm25 = None
        self.doc_mapping = None
        self.logger = logger

    def create_vector_store(self, documents: List[Document], vector_store_path: str):
        """
        문서로부터 FAISS 벡터스토어를 생성.
        """
        self.logger.info("FAISS 벡터스토어 생성 중...")
        
        # 문서 임베딩
        embeddings = np.array([self.embeddings.embed_query(doc.page_content) for doc in documents], dtype="float32")
        
        # FAISS 인덱스 생성
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        
        # 저장
        os.makedirs(vector_store_path, exist_ok=True)
        faiss.write_index(index, os.path.join(vector_store_path, "index.faiss"))
        
        # 문서 매핑 저장
        with open(os.path.join(vector_store_path, "metadata.pkl"), "wb") as f:
            pickle.dump(documents, f)
        
        self.logger.info("FAISS 벡터스토어 생성 완료.")
        self.vector_store = index
        self.doc_mapping = {i: doc for i, doc in enumerate(documents)}

    def load_vector_store(self, vector_store_path: str):
        """
        저장된 FAISS 벡터스토어와 문서 매핑 로드.
        """
        try:
            self.logger.info(f"FAISS 벡터스토어를 {vector_store_path}에서 로드합니다...")
            self.vector_store = faiss.read_index(os.path.join(vector_store_path, "index.faiss"))
            
            with open(os.path.join(vector_store_path, "metadata.pkl"), "rb") as f:
                documents = pickle.load(f)
                self.doc_mapping = {i: doc for i, doc in enumerate(documents)}
            
            self.logger.info("FAISS 벡터스토어 로드 완료.")
        except Exception as e:
            self.logger.error(f"로드 실패: {str(e)}")
            raise

    def initialize_bm25(self, documents: List[Document]):
        """
        BM25 검색 엔진 초기화.
        """
        self.logger.info("BM25 검색 엔진 초기화 중...")
        tokenized_corpus = [doc.page_content.lower().split() for doc in documents]
        self.bm25 = BM25Okapi(tokenized_corpus)
        self.logger.info("BM25 검색 엔진 초기화 완료.")

    def semantic_search(self, query: str, k: int = 5) -> List[Tuple[Document, float]]:
        """
        FAISS 벡터스토어를 활용한 의미론적 검색.
        """
        query_embedding = np.array([self.embeddings.embed_query(query)], dtype="float32")
        distances, indices = self.vector_store.search(query_embedding, k)
        results = [(self.doc_mapping[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
        return results

    def keyword_search(self, query: str, k: int = 5) -> List[Tuple[Document, float]]:
        """
        BM25를 활용한 키워드 검색.
        """
        if not self.bm25:
            raise ValueError("BM25 검색 엔진이 초기화되지 않았습니다.")
        
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        top_k_indices = np.argsort(scores)[-k:][::-1]
        results = [(self.doc_mapping[idx], scores[idx]) for idx in top_k_indices]
        return results

    def hybrid_search(self, query: str, k: int = 5, semantic_weight: float = 0.5) -> List[Tuple[Document, float]]:
        """
        의미론적 검색과 키워드 검색을 결합한 하이브리드 검색.
        """
        semantic_results = self.semantic_search(query, k)
        keyword_results = self.keyword_search(query, k)
        
        combined_scores = {}
        
        for doc, score in semantic_results:
            doc_id = id(doc)
            combined_scores[doc_id] = {'doc': doc, 'score': semantic_weight * (1 - score)}
        
        for doc, score in keyword_results:
            doc_id = id(doc)
            if doc_id in combined_scores:
                combined_scores[doc_id]['score'] += (1 - semantic_weight) * score
            else:
                combined_scores[doc_id] = {'doc': doc, 'score': (1 - semantic_weight) * score}
        
        sorted_results = sorted(
            [(info['doc'], info['score']) for info in combined_scores.values()],
            key=lambda x: x[1],
            reverse=True
        )[:k]
        
        return sorted_results


In [27]:
# 환경 변수에서 경로 가져오기
vector_store_path = "books_vectorstore"
books_dir = "books"
metadata_path = "books_vectorstore/index.pkl"

# 임베딩 모델 초기화 
embedding_model = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

# RAG 시스템 초기화
rag = AIBooksRAG(embedding_model)

try:
    # 기존 벡터 스토어 로드 시도
    rag.load_vector_store(vector_store_path, metadata_path)
    print("✅ 기존 벡터 스토어를 로드했습니다.")
    
except Exception as e:
    print(f"벡터 스토어 로드 실패: {str(e)}")
    raise

# 대화형 검색 시작
print("\n🔍 AI 뉴스 검색 시스템을 시작합니다.")
print("- 종료하려면 'q' 또는 'quit'를 입력하세요.")

search_mode = "hybrid" # 검색 방식 변경은 'mode [semantic/keyword/hybrid]'를 입력하세요.
while True:
    query = input("\n🔍 검색할 내용을 입력하세요: ").strip()

    if not query:
        continue
        
    if query.lower() in ['q', 'quit']:
        print("\n👋 검색을 종료합니다.")
        break
        
    if query.lower().startswith('mode '):
        mode = query.split()[1].lower()
        if mode in ['semantic', 'keyword', 'hybrid']:
            search_mode = mode
            print(f"\n✅ 검색 모드를 '{mode}'로 변경했습니다.")
        else:
            print("\n❌ 잘못된 검색 모드입니다. semantic/keyword/hybrid 중 선택하세요.")
        continue

    try:
        print(f"\n'{query}' 검색을 시작합니다... (모드: {search_mode})")
        
        if search_mode == "hybrid":
            results = rag.hybrid_search(query, k=5, semantic_weight=0.5)
        elif search_mode == "semantic":
            results = rag.vector_store.similarity_search_with_score(query, k=5)
        else:  # keyword
            results = rag.keyword_search(query, k=5)
        
        print(f"\n✨ 검색 완료! {len(results)}개의 결과를 찾았습니다.\n")
        
        # 결과 출력
        for i, result in enumerate(results, 1):
            doc = result[0]
            score = result[1]
            print(f"\n{'='*80}")
            print(f"검색 결과 {i}/{len(results)}")
            print(f"제목: {doc.metadata['title']}")
            print(f"날짜: {doc.metadata['date']}")
            if search_mode == "hybrid":
                print(f"통합 점수: {score:.4f}")
            elif search_mode == "semantic":
                print(f"유사도 점수: {1 - (score/2):.4f}")
            else:
                print(f"BM25 점수: {score:.4f}")
            print(f"URL: {doc.metadata['url']}")
            print(f"{'-'*40}")
            print(f"내용:\n{doc.page_content[:300]}...")
            
    except Exception as e:
        print(f"\n❌ 검색 중 오류가 발생했습니다: {str(e)}")

2025-01-13 09:59:57,561 - 데이터를 books_vectorstore에서 로드합니다...
2025-01-13 09:59:57,563 - 로드 중 오류 발생: too many values to unpack (expected 2)
벡터 스토어 로드 실패: too many values to unpack (expected 2)


ValueError: too many values to unpack (expected 2)